In [1]:
global Path
if sc.master[0:5] == "local":
    Path = "file:/home/hduser/pythonwork/PythonProject/"
else:
    Path="hdfs://master:9000/user/hduser/"

In [2]:
rawData=sc.textFile(Path+"data/u.data")
rawData.count()
rawRatings = rawData.map(lambda x: x.split('\t'))
rawRatings.take(5)
#数据分别是userId，itemId，rating和timestamp

[[u'196', u'242', u'3', u'881250949'],
 [u'186', u'302', u'3', u'891717742'],
 [u'22', u'377', u'1', u'878887116'],
 [u'244', u'51', u'2', u'880606923'],
 [u'166', u'346', u'1', u'886397596']]

In [3]:
from pyspark.mllib.recommendation import Rating, ALS
ratings = rawRatings.map(lambda x: Rating(int(x[0]),int(x[1]),float(x[2])))
print ratings.take(5)

[Rating(user=196, product=242, rating=3.0), Rating(user=186, product=302, rating=3.0), Rating(user=22, product=377, rating=1.0), Rating(user=244, product=51, rating=2.0), Rating(user=166, product=346, rating=1.0)]


In [4]:
model = ALS.train(ratings, 50, 10, 0.01)
userFeatures = model.userFeatures()
print userFeatures.take(2)

[(1, array('d', [-0.443621963262558, 0.15249183773994446, 0.00809907354414463, -0.03659500181674957, -0.30182206630706787, 0.06949819624423981, 0.5772250890731812, 0.1486252397298813, 0.13663265109062195, -0.08983384072780609, 0.11645139753818512, -0.2169293314218521, -0.20583516359329224, -0.17131078243255615, -0.24830088019371033, 0.3039054572582245, 0.7691220641136169, -0.041732169687747955, -0.09908940643072128, -0.19435235857963562, 0.16314193606376648, 0.2806546986103058, -0.42302682995796204, 0.22267131507396698, 0.20781876146793365, -0.8825811147689819, -0.4703618884086609, -0.4223354458808899, 0.2070450782775879, 0.14437542855739594, -0.23625843226909637, 0.2147342413663864, 0.15590961277484894, 0.17372895777225494, -0.21592241525650024, -0.42611828446388245, -0.6725453734397888, 0.585101842880249, 1.0138194561004639, -0.16314949095249176, 0.03775069862604141, 0.02394452691078186, -0.32906123995780945, -0.35081127285957336, 0.13963407278060913, 0.32947900891304016, -0.14239943

In [5]:
print model.userFeatures().count()
print model.productFeatures().count()

943
1682


In [6]:
print len(userFeatures.first()[1])
predictRating = model.predict(789,123)
print predictRating

50
4.31839875801


In [7]:
topKRecs = model.recommendProducts(789,10)
print '给用户userId推荐其喜欢的item：'
for rec in topKRecs:
    print rec

给用户userId推荐其喜欢的item：
Rating(user=789, product=172, rating=5.19974443214813)
Rating(user=789, product=346, rating=5.162575607683306)
Rating(user=789, product=185, rating=5.142733419050042)
Rating(user=789, product=192, rating=5.133010997968043)
Rating(user=789, product=182, rating=5.054696635700444)
Rating(user=789, product=176, rating=5.040063007047221)
Rating(user=789, product=127, rating=5.0355258478983025)
Rating(user=789, product=475, rating=5.014262470018285)
Rating(user=789, product=276, rating=4.9788931075075595)
Rating(user=789, product=129, rating=4.966722970892586)


In [8]:
moviesForUser = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(789)
print '用户对%d部电影进行了评级'%len(moviesForUser[0])
print '源数据中用户(userId=789)喜欢的电影(item)：'
for i in sorted(moviesForUser[0],key=lambda x : x.rating,reverse=True):
    print i.product

用户对33部电影进行了评级
源数据中用户(userId=789)喜欢的电影(item)：
127
475
9
50
150
276
129
100
741
1012
93
293
181
1008
508
1007
124
1161
294
1
284
1017
111
742
248
249
591
288
762
628
137
151
286


In [10]:
movies = sc.textFile(Path+"data/u.item")
titles = movies.map(lambda line: (int(line.split('|')[0]),line.split('|')[1])).collectAsMap()
print titles[1]

Toy Story (1995)


In [11]:
for i,rec in enumerate(topKRecs):
    print 'rank:'+str(i)+' '+str(titles[rec.product])+':'+str(rec.rating)

rank:0 Empire Strikes Back, The (1980):5.19974443215
rank:1 Jackie Brown (1997):5.16257560768
rank:2 Psycho (1960):5.14273341905
rank:3 Raging Bull (1980):5.13301099797
rank:4 GoodFellas (1990):5.0546966357
rank:5 Aliens (1986):5.04006300705
rank:6 Godfather, The (1972):5.0355258479
rank:7 Trainspotting (1996):5.01426247002
rank:8 Leaving Las Vegas (1995):4.97889310751
rank:9 Bound (1996):4.96672297089


In [12]:
import numpy as np
def cosineSImilarity(x,y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
testx = np.array([1.0,2.0,3.0])
print cosineSImilarity(testx,testx)

1.0


In [13]:
itemId = 567
itemFactor = model.productFeatures().lookup(itemId)[0]
print itemFactor

#print model.productFeatures().collect()
#print model.productFeatures().take(1)
sims = model.productFeatures().map(lambda (id,factor):(id,cosineSImilarity(np.array(factor),
             np.array(itemFactor))))

sims.sortBy(lambda (x,y):y,ascending=False).take(10)

array('d', [-0.12362880259752274, 0.3386056423187256, -0.05198248103260994, -0.053988974541425705, 0.3720850646495819, 0.40340402722358704, -0.18809355795383453, -0.37115129828453064, -0.39677128195762634, -0.005402594804763794, 0.6166006922721863, -0.47819486260414124, 0.8201115727424622, 0.6075004935264587, 0.11704376339912415, 1.5614676475524902, 0.24599185585975647, 0.5857929587364197, -0.25336727499961853, -0.2535555362701416, 0.5496000051498413, 0.6936941146850586, -0.5321304202079773, 0.4159337282180786, -0.3149735927581787, -0.1478659212589264, -0.4978523254394531, 0.29492443799972534, -0.5457490086555481, 0.026110554113984108, -1.4289947748184204, 1.3125420808792114, 0.6412392258644104, 0.6195180416107178, -0.18011830747127533, -0.5842649340629578, -0.2692536413669586, -0.19549345970153809, 0.9430791139602661, -0.20924484729766846, 1.31484055519104, -0.2473936378955841, 0.37797924876213074, -0.4720618426799774, -0.686421811580658, 0.14603674411773682, 0.4873642921447754, 0.367

[(567, 1.0000000000000002),
 (563, 0.75304166406621798),
 (685, 0.71801557440311381),
 (1376, 0.71462491264225225),
 (201, 0.70848338200709882),
 (781, 0.70091329017855053),
 (945, 0.69643413906109497),
 (1046, 0.68710312504518978),
 (670, 0.68448189415426175),
 (741, 0.67796929627869507)]

In [32]:
usersFeatures = model.userFeatures()
print usersFeatures.take(2) 

[(2, array('d', [0.5532510876655579, 1.3408938646316528, -0.0009969864040613174, 0.15598875284194946, 0.015765467658638954, 1.25249183177948, 0.26990532875061035, -0.2956365644931793, 0.8075549006462097, -0.39656248688697815])), (4, array('d', [1.240945816040039, 0.5230079889297485, -0.8210291266441345, -1.2046180963516235, -0.46608495712280273, 0.8748364448547363, -0.403978168964386, -0.6052988767623901, -0.28171080350875854, -1.1605303287506104]))]


In [14]:
actual = moviesForUser[0][0]
actualRating = actual.rating
print '用户789对电影1012的实际评级',actualRating
predictedRating = model.predict(789, actual.product)
print '用户789对电影1012的预测评级',predictedRating
squaredError = np.power(actualRating-predictedRating,2)
print '实际评级与预测评级的MSE',squaredError

用户789对电影1012的实际评级 4.0
用户789对电影1012的预测评级 3.93902267122
实际评级与预测评级的MSE 0.00371823462522


In [15]:
moviesForUser

[[Rating(user=789, product=1012, rating=4.0),
  Rating(user=789, product=127, rating=5.0),
  Rating(user=789, product=475, rating=5.0),
  Rating(user=789, product=93, rating=4.0),
  Rating(user=789, product=1161, rating=3.0),
  Rating(user=789, product=286, rating=1.0),
  Rating(user=789, product=293, rating=4.0),
  Rating(user=789, product=9, rating=5.0),
  Rating(user=789, product=50, rating=5.0),
  Rating(user=789, product=294, rating=3.0),
  Rating(user=789, product=181, rating=4.0),
  Rating(user=789, product=1, rating=3.0),
  Rating(user=789, product=1008, rating=4.0),
  Rating(user=789, product=508, rating=4.0),
  Rating(user=789, product=284, rating=3.0),
  Rating(user=789, product=1017, rating=3.0),
  Rating(user=789, product=137, rating=2.0),
  Rating(user=789, product=111, rating=3.0),
  Rating(user=789, product=742, rating=3.0),
  Rating(user=789, product=248, rating=3.0),
  Rating(user=789, product=249, rating=3.0),
  Rating(user=789, product=1007, rating=4.0),
  Rating(us

In [16]:
userProducts = ratings.map(lambda rating:(rating.user,rating.product))
print '实际的评分:',userProducts.take(5)

#预测所有用户对电影的相应评分
print model.predictAll(userProducts).collect()[0]
predictions = model.predictAll(userProducts).map(lambda rating:((rating.user,rating.product), rating.rating))
print '预测的评分:',predictions.take(5)

ratingsAndPredictions = ratings.map(lambda rating:((rating.user,rating.product),rating.rating)).join(predictions)
print '组合预测的评分和实际的评分:',ratingsAndPredictions.take(5)

MSE = ratingsAndPredictions.map(lambda ((x,y),(m,n)):np.power(m-n,2)).reduce(lambda x,y:x+y)/ratingsAndPredictions.count() 
print '模型的均方误差:',MSE 
print '模型的均方根误差:',np.sqrt(MSE)

实际的评分: [(196, 242), (186, 302), (22, 377), (244, 51), (166, 346)]
Rating(user=195, product=1084, rating=3.908824136441636)
预测的评分: [((195, 1084), 3.908824136441636), ((58, 1084), 3.9570059102930157), ((541, 1084), 4.069377274374024), ((470, 1084), 2.9511299331553356), ((682, 1084), 2.0420265669668636)]
组合预测的评分和实际的评分: [((621, 577), (3.0, 3.0573340695101994)), ((877, 727), (4.0, 3.931995198915532)), ((109, 365), (4.0, 3.9696760588648288)), ((254, 622), (4.0, 3.817703061427666)), ((720, 286), (5.0, 4.954397877466782))]
模型的均方误差: 0.0830304803697
模型的均方根误差: 0.288150100416


In [17]:
def avgPrecisionK(actual, predicted, k): 
    if len(predicted) > k:
        predK = predicted[:k]
    else:
        predK = predicted
    score = 0.0
    numHits = 0.0
    for i,p in enumerate(predK):
        if p in actual and p not in predK:
            numHits = numHits + 1
            score = score + numHits/(i+1)
    if not actual:
        return 1.0
    else:
        return score/min(len(actual),k)

In [18]:
actualMovies = [rating.product for rating in moviesForUser[0]]
predictMovies = [rating.product for rating in topKRecs]
print '实际的电影：',actualMovies
print '预测的电影：',predictMovies

实际的电影： [1012, 127, 475, 93, 1161, 286, 293, 9, 50, 294, 181, 1, 1008, 508, 284, 1017, 137, 111, 742, 248, 249, 1007, 591, 150, 276, 151, 129, 100, 741, 288, 762, 628, 124]
预测的电影： [172, 346, 185, 192, 182, 176, 127, 475, 276, 129]


In [19]:
MAP10 = avgPrecisionK(actualMovies,predictMovies,10)
print MAP10

0.0


In [20]:
itemFactors = model.productFeatures().map(lambda (id,factor):factor).collect()
itemMatrix = np.array(itemFactors)
print itemMatrix
print itemMatrix.shape

[[ 0.03003818  0.61005521  0.19261409 ..., -0.90471607 -0.61920935
  -0.22159728]
 [ 0.084612    0.30667374  0.33825687 ..., -0.33154243  0.03173169
   0.0391243 ]
 [ 0.06009899  0.84202212 -0.05745131 ..., -0.08486437 -0.45201391
   0.35717988]
 ..., 
 [ 0.1275907  -0.01938788  0.02511401 ..., -0.10829373 -0.0651757
  -0.01354955]
 [-0.01374614 -0.09747539  0.01012929 ..., -0.08876308 -0.1251747
   0.03300911]
 [ 0.10422719  0.15160774  0.19717528 ..., -0.27291617  0.0303707
  -0.02773944]]
(1682, 50)


In [21]:
#便于后面进行map计算的时候使所有map都有效，这里在实际当中的意思是把数据广播到任何一个节点
imBroadcast = sc.broadcast(itemMatrix)

In [22]:
userVector = model.userFeatures().map(lambda (userId,array):(userId,np.array(array)))
#print userVector.collect()[0][1].shape
userVector = userVector.map(lambda (userId,x): (userId,imBroadcast.value.dot((np.array(x).transpose()))))
#print userVector.collect()[0][1].shape
userVectorId = userVector.map(lambda (userId,x) : (userId,[(xx,i) for i,xx in enumerate(x.tolist())]))
#print userVectorId.collect()[0]
sortUserVectorId = userVectorId.map(lambda (userId,x):(userId,sorted(x,key=lambda x:x[0],reverse=True)))
sortUserVectorRecId = sortUserVectorId.map(lambda (userId,x): (userId,[xx[1] for xx in x]))
#print sortUserVectorRecId.take(2)
sortUserVectorRecId.count()

943

In [23]:
userMovies = ratings.map(lambda rating: (rating.user,rating.product)).groupBy(lambda (x,y):x)
print userMovies.take(3)
userMovies = userMovies.map(lambda (userId,x):(userId, [xx[1] for xx in x] ))
#print userMovies.take(1)
#allAPK=sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):(userId,avgPrecisionK(actual,predicted,10)))
#print allAPK.map(lambda (x,y):y).reduce(lambda x,y:x+y)/sortUserVectorRecId.count()
allAPK=sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):avgPrecisionK(actual,predicted,2000))
# print allAPK.take(10)
print allAPK.reduce(lambda x,y:x+y)/allAPK.count()

[(1, <pyspark.resultiterable.ResultIterable object at 0x7fe558835f90>), (2, <pyspark.resultiterable.ResultIterable object at 0x7fe5588350d0>), (3, <pyspark.resultiterable.ResultIterable object at 0x7fe559d4d3d0>)]
0.0


In [24]:
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.evaluation import RankingMetrics
predictedAndTrue = ratingsAndPredictions.map(lambda ((userId,product),(predicted, actual))
      :(predicted,actual))
print predictedAndTrue.take(5)
regressionMetrics = RegressionMetrics(predictedAndTrue)
print "均方误差 = %f"%regressionMetrics.meanSquaredError
print "均方根误差 = %f"% regressionMetrics.rootMeanSquaredError

[(3.0, 3.0573340695101994), (4.0, 3.931995198915532), (4.0, 3.9696760588648288), (4.0, 3.817703061427666), (5.0, 4.954397877466782)]
均方误差 = 0.083030
均方根误差 = 0.288150


In [25]:
sortedLabels = sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):(predicted,actual))
# print sortedLabels.take(1)
rankMetrics = RankingMetrics(sortedLabels)
print "Mean Average Precision = %f" % rankMetrics.meanAveragePrecision
print "Mean Average Precision(at K=10) = %f" % rankMetrics.precisionAt(10)

Mean Average Precision = 0.117916
Mean Average Precision(at K=10) = 0.138812
